# Python visualization

date = Oktober 18, 2020 <br>
author = c.magg <br>

In [1]:
import os
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt

In [2]:
sys.path.append(os.path.abspath('../../'))

In [3]:
import vtk
from vtk.qt.QVTKRenderWindowInteractor import QVTKRenderWindowInteractor

In [4]:
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QToolBar 
from PyQt5.QtWidgets import QStatusBar, QCheckBox, QAction
from PyQt5.QtWidgets import QFrame, QVBoxLayout, QWidget
from PyQt5.QtWidgets import QPushButton, QFileDialog, QSlider
from PyQt5.QtCore import Qt

In [5]:
from slice_text import SliceText

In [6]:
path_dir = "../../Data/1/Segmentation"
time_steps = [x for x in os.listdir(path_dir) if 'png' in x]
time_steps

['png', 'png 3-15-16', 'png 3-23-16', 'png 4-29-16']

In [7]:
path_dicom_dir = "../../Data/1/Segmentation/png"
path_dicom_files = [os.path.join(path_dicom_dir,x) for x in os.listdir(path_dicom_dir)]
#sorted(path_dicom_files, key=lambda x: int(x.split('slice')[-1].split('.')[0]))

In [8]:
class TimeText():
    
    def __init__(self, current_time_step, max_time_step):
        # Time step status message
        self.timeTextProp = vtk.vtkTextProperty()
        self.timeTextProp.SetFontFamilyToCourier()
        self.timeTextProp.SetFontSize(20)
        self.timeTextProp.SetColor(128, 128, 128)
        self.timeTextProp.SetOpacity(1)
        self.timeTextProp.SetVerticalJustificationToBottom()
        self.timeTextProp.SetJustificationToLeft()

        self.timeTextMapper = vtk.vtkTextMapper()
        message = self.status_message(current_time_step, max_time_step)
        self.timeTextMapper.SetInput(message)
        self.timeTextMapper.SetTextProperty(self.timeTextProp)

        self.timeTextActor = vtk.vtkActor2D()
        self.timeTextActor.SetMapper(self.timeTextMapper)
        self.timeTextActor.SetPosition(15, 30)

    def SetInput(self, current_time_step, max_time_step):
        message = self.status_message(current_time_step, max_time_step)
        self.timeTextMapper.SetInput(message)

    def status_message(self, current_time_step, max_time_step):
        return "Time {0} / {1}".format(current_time_step, max_time_step)

In [9]:
class HelpingWindow(QMainWindow):
    
    def __init__(self, *args, **kwargs):
        super(HelpingWindow, self).__init__(*args, **kwargs)
        self.setWindowTitle("Navigation")
        
        widget = QWidget(self)
        self.setCentralWidget(widget)
        layout = QVBoxLayout(widget)
        
        info = QLabel(
            "left mouse button - change window level\n"+
            "right left mouse - positional zoom\n"+
            "mouse wheele - slice through volume\n")
        info.setAlignment(Qt.AlignLeft | Qt.AlignVCenter)
        layout.addWidget(info)
        
        #quit = QAction("Quit",self) 
        #quit.triggered.connect(self.closeWindow)
        #layout.addAction(quit)
        
    def closeWindow(self, s):
        self.close()

In [10]:
class VTKPipeline:
    
    def __init__(self, nr_time_steps):        
        
        self.window = None
        
        # Read data
        self.reader = vtk.vtkPNGReader()
        path_dicom_dir = "../../Data/1/Segmentation/png"
        path_dicom_files = [os.path.join(path_dicom_dir,x) for x in os.listdir(path_dicom_dir)]
        path_dicom_files = sorted(path_dicom_files, key=lambda x: int(x.split('slice')[-1].split('.')[0]))
        self.UpdateReader(path_dicom_files)
        
        self.dicom = vtk.vtkImageSlice()
        self.dicom.SetMapper(vtk.vtkImageSliceMapper())
        self.dicom.GetMapper().SetSliceNumber(10)
        self.dicom.GetProperty().SetOpacity(0.5)
        self.dicom.GetMapper().SetInputConnection(self.reader.GetOutputPort())
               
        # Slice status message
        slice_number = self.dicom.GetMapper().GetSliceNumber()
        slice_number_max = self.dicom.GetMapper().GetSliceNumberMaxValue()
        self.sliceText = SliceText(slice_number, slice_number_max)
        
        # Time status message
        self.timeText = TimeText(0, nr_time_steps)
        
        # create and add renderer
        self.renderer = vtk.vtkRenderer() 
        self.renderer.SetBackground(0,0,0)
        self.renderer.ResetCamera()         
        self.renderer.AddViewProp(self.dicom)
        self.renderer.AddActor2D(self.sliceText.sliceTextActor)
        self.renderer.AddActor2D(self.timeText.timeTextActor)
        
        # Create interactor (customized)
        self.interactorStyle = vtk.vtkInteractorStyleImage()
        self.interactorStyle.SetInteractionModeToImageSlicing()
        # Create interactor
        self.interactor = vtk.vtkRenderWindowInteractor()
        self.interactor.SetInteractorStyle(self.interactorStyle)
        self.interactorStyle.AddObserver("MouseWheelForwardEvent", self.MoveSliceFoward)
        self.interactorStyle.AddObserver("MouseWheelBackwardEvent", self.MoveSliceBackward)
        
    def RemoveDicomActor(self):
        self.renderer.RemoveViewProp(self.dicom)
        self.window.Render()
        
    def AddDicomActor(self):
        self.renderer.AddViewProp(self.dicom)
        self.window.Render()
                  
    def UpdateReader(self, new_path):
        filePath = vtk.vtkStringArray()
        filePath.SetNumberOfValues(len(new_path))
        for i in range(0,len(new_path),1):
            filePath.SetValue(i, new_path[i])
        print("first file name ", new_path[0])
        self.reader.SetFileNames(filePath)
        self.reader.Update()
        
    def SetWindow(self, window):
        self.window = window
    
    def MoveSliceFoward(self, obj, event):
        current_slice = self.dicom.GetMapper().GetSliceNumber()
        self.MoveSlice(current_slice + 1)

    def MoveSliceBackward(self, obj, event):
        current_slice = self.dicom.GetMapper().GetSliceNumber()
        self.MoveSlice(current_slice - 1)

    def MoveSlice(self, new_slice):
        new_slice = max(min(self.dicom.GetMapper().GetSliceNumberMaxValue(), new_slice),
                        self.dicom.GetMapper().GetSliceNumberMinValue())
        print('setting new slice', new_slice)
        self.dicom.GetMapper().SetSliceNumber(new_slice)
        slice_number = self.dicom.GetMapper().GetSliceNumber()
        self.sliceText.SetInput(new_slice, self.dicom.GetMapper().GetSliceNumberMaxValue())
        self.window.Render()
        
    def SetTimeText(self, new_time_step, max_time_step):
        print("setting new time step ", new_time_step)
        self.timeText.SetInput(new_time_step, max_time_step)
        self.window.Render()

In [11]:
class MainWindow(QMainWindow):

    def __init__(self, switch = True, parent = None):
        QMainWindow.__init__(self, parent)
        
        init_time_step = 0
        self.switch = switch
              
        # Set up frame
        self.frame = QFrame()
        self.layout = QVBoxLayout()
                              
         # VTK widget
        self.vtkWidget = QVTKRenderWindowInteractor(self.frame)
        self.layout.addWidget(self.vtkWidget)
        
        self.path_dir = "../../Data/1/Segmentation/"
        self.nr_time_steps = len([x for x in os.listdir(self.path_dir) if 'png' in x])-1
        self.vtkPipeline = VTKPipeline(self.nr_time_steps)
        self.changeReader(nr_time_step=0)
        self.vtkPipeline.SetWindow(self.vtkWidget)
        self.renderer = self.vtkPipeline.renderer
        self.vtkWidget.GetRenderWindow().AddRenderer(self.renderer)
        self.interactor = self.vtkWidget.GetRenderWindow().GetInteractor() 
        self.interactor.SetInteractorStyle(self.vtkPipeline.interactorStyle)
        
        self.frame.setLayout(self.layout)
        self.setCentralWidget(self.frame)
        
        # Toolbar with Slider
        self.toolbar = QToolBar("Time slider") 
        self.addToolBar(self.toolbar)
        self.createSliderToolbar()        
        
        # Menu Bar
        bar = self.menuBar()        
        new_folder = QAction("New Patient",self)
        new_folder.triggered.connect(self.openFolder)
        #new_folder.setShortcut("Ctrl+N")
        bar.addAction(new_folder)
              
        helping = QAction("Help", self)
        helping.triggered.connect(self.helping_button)
        bar.addAction(helping)
        
        quit = QAction("Quit",self) 
        quit.triggered.connect(self.closeWindow)
        bar.addAction(quit)
        
        # Show and initialize
        self.show()
        self.vtkWidget.Initialize()
        
    def closeWindow(self, s):
        self.close()
        
    def createSliderToolbar(self, init_value=0):
        self.removeToolBar(self.toolbar)
        self.toolbar = QToolBar("Time slider") 
        
        self.slider = QSlider(Qt.Horizontal)
        self.slider.valueChanged.connect(self.timeStepChange)
        self.slider.setMinimum(0)
        self.slider.setMaximum(len(self.time_steps)-1)
        if init_value > len(self.time_steps)-1:
            init_value = len(self.time_steps)-1
        self.slider.setValue(init_value)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(1)    
        print('Create slider {0}/{1}'.format(init_value, len(self.time_steps)-1))
        self.toolbar.addWidget(self.slider)
        
        self.toolbar.addSeparator()
        
        background_toggle = QCheckBox()
        background_toggle.setText("Background Toggle")
        background_toggle.setChecked(True)
        background_toggle.stateChanged.connect(self.backgroundToogleChange)
        
        self.toolbar.addWidget(background_toggle)
        self.addToolBar(self.toolbar)
        
    def backgroundToogleChange(self, s):
        pass
        if s == 0:
            print('no background')
            self.vtkPipeline.RemoveDicomActor()
        elif s == 2:
            print('background')
            self.vtkPipeline.AddDicomActor()
        
    def openFolder(self, s):        
        file_dialog = QFileDialog()
        file_dialog.setFileMode(QFileDialog.DirectoryOnly)
        file_dialog.setViewMode(QFileDialog.Detail)
        
        if file_dialog.exec_():
            self.path_dir = file_dialog.selectedFiles()[0]
            print('Open folder', self.path_dir)
            self.changeReader()
            self.createSliderToolbar(init_value=self.slider.value())
            
    def changeReader(self, nr_time_step=None):
        self.time_steps = [x for x in os.listdir(self.path_dir) if 'png' in x]
        self.nr_time_steps = len(self.time_steps)-1
        #print(self.time_steps)
        if nr_time_step is None:
            nr_time_step = min(self.nr_time_steps,self.slider.value())
            self.slider.setValue(nr_time_step)
            self.vtkPipeline.SetTimeText(self.slider.value(), self.nr_time_steps)
        path_dicom_dir = os.path.join(self.path_dir, self.time_steps[nr_time_step]) 
        path_dicom_files = [os.path.join(path_dicom_dir,x) for x in os.listdir(path_dicom_dir)]
        path_dicom_files = sorted(path_dicom_files, key=lambda x: int(x.split('slice')[-1].split('.')[0]))
        self.vtkPipeline.UpdateReader(path_dicom_files)
        print('Change folder', path_dicom_dir)
        
    def helping_button(self):
        self.helpingWindow = HelpingWindow()
        self.helpingWindow.show()
            
    def timeStepChange(self,s):
        print('Change step {0}/{1}'.format(self.slider.value(),self.nr_time_steps))
        self.vtkPipeline.SetTimeText(self.slider.value(), self.nr_time_steps)
        self.changeReader()       

In [12]:
app = QApplication(sys.argv)
window = MainWindow(True)
window.show()

app.exec_()

first file name  ../../Data/1/Segmentation/png\1_slice0.png
first file name  ../../Data/1/Segmentation/png\1_slice0.png
Change folder ../../Data/1/Segmentation/png
Create slider 0/3
setting new slice 11
setting new slice 12
setting new slice 13
setting new slice 14
setting new slice 15
setting new slice 16
setting new slice 17
setting new slice 18
setting new slice 19
setting new slice 20
setting new slice 21
setting new slice 22
setting new slice 23
setting new slice 24
setting new slice 25
setting new slice 26
setting new slice 27
setting new slice 28
setting new slice 29
setting new slice 30
setting new slice 31
setting new slice 32
setting new slice 33
Change step 1/3
setting new time step  1
setting new time step  1
first file name  ../../Data/1/Segmentation/png 3-15-16\1_slice0.png
Change folder ../../Data/1/Segmentation/png 3-15-16
Change step 2/3
setting new time step  2
setting new time step  2
first file name  ../../Data/1/Segmentation/png 3-23-16\1_slice0.png
Change folder .

Change folder C:/Users/Caroline/Documents/KidsBrainProject/Data/5/Segmentation\png 12-23-16
Change step 0/2
setting new time step  0
setting new time step  0
first file name  C:/Users/Caroline/Documents/KidsBrainProject/Data/5/Segmentation\png\5_slice0.png
Change folder C:/Users/Caroline/Documents/KidsBrainProject/Data/5/Segmentation\png
Open folder C:/Users/Caroline/Documents/KidsBrainProject/Data/6/Segmentation
setting new time step  0
first file name  C:/Users/Caroline/Documents/KidsBrainProject/Data/6/Segmentation\png\6_slice0.png
Change folder C:/Users/Caroline/Documents/KidsBrainProject/Data/6/Segmentation\png
Create slider 0/2
Change step 2/2
setting new time step  2
setting new time step  2
first file name  C:/Users/Caroline/Documents/KidsBrainProject/Data/6/Segmentation\png 12-1-16\6_slice0.png
Change folder C:/Users/Caroline/Documents/KidsBrainProject/Data/6/Segmentation\png 12-1-16
Change step 1/2
setting new time step  1
setting new time step  1
first file name  C:/Users/Ca

0

In [13]:
window.vtkWidget.GetRenderWindow().GetInteractor()

(vtkRenderingCorePython.vtkGenericRenderWindowInteractor)000000B0C9D82648

In [14]:
window.vtkWidget.Render

<bound method QVTKRenderWindowInteractor.Render of <vtk.qt.QVTKRenderWindowInteractor.QVTKRenderWindowInteractor object at 0x000000B0C9DC7708>>